# 1) Introdução

O intuito do presente trabalho é desenvolver um protótipo de uma plataforma que compile métricas sociais de organizações listadas na bolsa brasileira, promovendo assim, uma maior transparência para investidores conscientes tomarem suas decisões. 

Dentro do desafio proposto, cujo objetivo é responder à pergunta *"Como a tecnologia pode ajudar investidores, gestores ou empresas a aderirem ao ESG de forma consciente e produtiva?"*, optamos por sintetizar em uma única plataforma métricas colhidas a partir de colaboradores e clientes das organizações, dando um score final a cada organização. Nosso intuito é dar uma maior visibilidade ao âmbito **Social** dentro do ESG e, para isso, desenvolvemos uma solução que irá quantificar o quanto uma organização está desenvolvida nas seguintes questões:

- **Políticas de diversidade e inclusão**: Serão coletados dados de funcionários da organização via web scrapping do Linkedin, a fim de quantificar a distribuição de gênero destes. Será desenvolvido um modelo de NLP que irá identificar o gênero do colaborador a partir do seu nome e, ao final, será criada uma métrica que irá aumentar o score final da organização quando essa distribuição entre colaboradores femininos e masculinos mais se aproximará de 50/50.

- **Qualidade do atendimento ao cliente**: Serão coletados os scores do ReclameAqui das organizações, a fim de identificar aquelas que possuem uma melhor política de atendimento.

- **Relações de trabalho**: Serão coletados os scores do Glassdoor, a fim de identificar aquelas que possuem um maior reconhecimento por parte dos seus colaboradores.

A nível de demonstração, foram escolhidas duas organizações do setor de varejo para que possamos comparar seus indicadores e inferirmos qual está mais avançada nas politica sociais do ESG, sendo elas a **Magazine Luiza** e a **Americanas S.A**.



# 2) Importação das bibliotecas

In [154]:
import pandas as pd
import numpy as np
import random

from nltk.corpus import names
from nltk.classify import apply_features

# 3) Políticas de diversidade e inclusão

## 3.1) Magazine Luiza

### 3.1.1) Importação e limpeza dos dados de colaboradores 

Primeiro, vamos importar o arquivo CSV contendo a lista de colaboradores da Magazine Luiza. A lista foi obtida por meio da plataforma Phantom Buster (https://phantombuster.com/), que utiliza web scrapping para retornar os valores em formato estruturado, dado um link contendo a pesquisa do Linkedin. Por limitações da própria plataforma, é permitido apenas o dowload de 1.000 registros por dia, de forma que teremos que trabalhar com apenas uma amostra.

In [157]:
nomes_ml = pd.read_csv("magalu.csv")
print("Shape: ", nomes_ml.shape)
print("Colunas: ", nomes_ml.columns)

Shape:  (999, 19)
Colunas:  Index(['profileUrl', 'fullName', 'firstName', 'lastName', 'profileImageUrl',
       'currentJob', 'pastJob', 'connectionDegree', 'job', 'location',
       'sharedConnections', 'commonConnection1', 'commonConnection2', 'url',
       'name', 'query', 'category', 'timestamp', 'error'],
      dtype='object')


In [158]:
nomes_ml.head()

,profileUrl,fullName,firstName,lastName,profileImageUrl,currentJob,pastJob,connectionDegree,job,location,sharedConnections,commonConnection1,commonConnection2,url,name,query,category,timestamp,error
0,https://www.linkedin.com/in/stefanie-joana-gar...,Stefanie Joana Garcia Cavalcanti,Stefanie,Joana Garcia Cavalcanti,https://media-exp1.licdn.com/dms/image/C4D03AQ...,NaN,NaN,2º,Coordenadora de Recrutamento e Seleção - Luiza...,"São Paulo, SP","Murilo Gun, Renata Corotti e 12 outras conexõe...",https://www.linkedin.com/in/ACoAAAAGiM4BmEwyac...,https://www.linkedin.com/in/ACoAAAGOiGEBL7SE9a...,https://www.linkedin.com/in/stefanie-joana-gar...,Stefanie Joana Garcia Cavalcanti,https://www.linkedin.com/search/results/people...,People,2021-09-26T15:25:32.519Z,NaN
1,https://www.linkedin.com/in/larissa-pedroso-01...,Larissa Pedroso,Larissa,Pedroso,https://media-exp1.licdn.com/dms/image/C5603AQ...,NaN,NaN,2º,Coordenadora de Gestão de Pessoas,"Curitiba, PR",Rodrigo Sarlo é uma conexão em comum,https://www.linkedin.com/in/ACoAAAT-0A0BfBrCl7...,NaN,https://www.linkedin.com/in/larissa-pedroso-01...,Larissa Pedroso,https://www.linkedin.com/search/results/people...,People,2021-09-26T15:25:32.519Z,NaN
2,https://www.linkedin.com/in/jaqueline-fernande...,Jaqueline Fernandes,Jaqueline,Fernandes,https://media-exp1.licdn.com/dms/image/C5603AQ...,NaN,NaN,2º,Coordenadora de Recrutamento e Seleção na Maga...,"São Paulo, SP","Rodrigo Sarlo, José Alves do Monte e 4 outras ...",https://www.linkedin.com/in/ACoAAAT-0A0BfBrCl7...,https://www.linkedin.com/in/ACoAAAdTjgEBhthWov...,https://www.linkedin.com/in/jaqueline-fernande...,Jaqueline Fernandes,https://www.linkedin.com/search/results/people...,People,2021-09-26T15:25:32.519Z,NaN
3,https://www.linkedin.com/in/michele-machado,Michele Machado Bernardo,Michele,Machado Bernardo,https://media-exp1.licdn.com/dms/image/C4E03AQ...,NaN,NaN,2º,Analista de Recrutamento e Seleção no Magazine...,"São Paulo, SP","Leo Cesar Melo, Rodrigo Sarlo e 4 outras conex...",https://www.linkedin.com/in/ACoAAATnNSYB9Cyi9t...,https://www.linkedin.com/in/ACoAAAT-0A0BfBrCl7...,https://www.linkedin.com/in/michele-machado,Michele Machado Bernardo,https://www.linkedin.com/search/results/people...,People,2021-09-26T15:25:32.519Z,NaN
4,https://www.linkedin.com/in/jamile-coelho-sale...,Jamile Coelho Sales,Jamile,Coelho Sales,https://media-exp1.licdn.com/dms/image/C4E03AQ...,NaN,NaN,3º+,Coordenadora de Gestão de Pessoas,"Salvador, BA",NaN,NaN,NaN,https://www.linkedin.com/in/jamile-coelho-sale...,Jamile Coelho Sales,https://www.linkedin.com/search/results/people...,People,2021-09-26T15:25:32.519Z,NaN


Para nós, só interessa a coluna contendo o primeiro nome do colaborador, portanto, podemos descartar as demais e excluir registros com valores nulos.

In [159]:
nomes_ml = nomes_ml[['firstName']].dropna()
print("Shape: ", nomes_ml.shape)
nomes_ml.head()

Shape:  (990, 1)


,firstName
0,Stefanie
1,Larissa
2,Jaqueline
3,Michele
4,Jamile


### 3.1.2) Criação do modelo de machine learning que irá fazer a classificação de gênero dos colaboradores a partir de seus nomes.

Para fazermos a classificação de gênero, iremos utilizar a biblioteca NLTK (https://www.nltk.org/) que contém pacotes voltados para processamento de linguagem natural (NLP). Como primeiro passo, iremos retornar uma lista contendo pares de valores de nomes e gêneros associados já embarcados no sistema interno do pacote.

In [161]:
Gender_names = ([(name, 'male') for name in names.words('male.txt')] +
[(name, 'female') for name in names.words('female.txt')])
random.shuffle(Gender_names)

In [162]:
Gender_names[:20]

[('Stephannie', 'female'),
 ('Rikki', 'male'),
 ('Melina', 'female'),
 ('Wilhelmine', 'female'),
 ('Whit', 'male'),
 ('Eleonora', 'female'),
 ('Nancee', 'female'),
 ('Bjorne', 'male'),
 ('Sheri', 'female'),
 ('Sonni', 'female'),
 ('Cindra', 'female'),
 ('Bert', 'female'),
 ('Thaddeus', 'male'),
 ('Jeffry', 'male'),
 ('Danica', 'female'),
 ('Ranna', 'female'),
 ('Lust', 'female'),
 ('Jeannette', 'female'),
 ('Brea', 'female'),
 ('Mia', 'female')]

A seguir, podemos contar a quantidade total de nomes cadastrados no arquivo interno da biblioteca, assim como a distribuição destes entre gêneros masculinos e femininos:

In [163]:
Gender_names[0:10] 
len(Gender_names)
Gender_names_g= [g for (n,g) in Gender_names]
Gender_names_m = Gender_names_g.count('male')
Gender_names_f = Gender_names_g.count('female')
print("Total Male: {} and Female: {} in dataset of size: {}.".format(Gender_names_m,Gender_names_f,len(Gender_names_g)))

Total Male: 2943 and Female: 5001 in dataset of size: 7944.


Para o nosso modelo, iremos utilizar um método que utiliza como parâmetros a primeira letra do nome, a última letra do nome e a contagem de letras totais do nome. Abaixo, criamos uma função que, para dado um nome qualquer, retorna esses valores.

In [164]:
def gender_features_az(name):
    features = {}
    features["firstletter"] = name[0].lower()
    features["lastletter"] = name[-1].lower()
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features["count(%s)" % letter] = name.lower().count(letter)
        features["has(%s)" % letter] = (letter in name.lower())
    return features

In [165]:
gender_features_az("Navi Capital")

{'firstletter': 'n',
 'lastletter': 'l',
 'count(a)': 3,
 'has(a)': True,
 'count(b)': 0,
 'has(b)': False,
 'count(c)': 1,
 'has(c)': True,
 'count(d)': 0,
 'has(d)': False,
 'count(e)': 0,
 'has(e)': False,
 'count(f)': 0,
 'has(f)': False,
 'count(g)': 0,
 'has(g)': False,
 'count(h)': 0,
 'has(h)': False,
 'count(i)': 2,
 'has(i)': True,
 'count(j)': 0,
 'has(j)': False,
 'count(k)': 0,
 'has(k)': False,
 'count(l)': 1,
 'has(l)': True,
 'count(m)': 0,
 'has(m)': False,
 'count(n)': 1,
 'has(n)': True,
 'count(o)': 0,
 'has(o)': False,
 'count(p)': 1,
 'has(p)': True,
 'count(q)': 0,
 'has(q)': False,
 'count(r)': 0,
 'has(r)': False,
 'count(s)': 0,
 'has(s)': False,
 'count(t)': 1,
 'has(t)': True,
 'count(u)': 0,
 'has(u)': False,
 'count(v)': 1,
 'has(v)': True,
 'count(w)': 0,
 'has(w)': False,
 'count(x)': 0,
 'has(x)': False,
 'count(y)': 0,
 'has(y)': False,
 'count(z)': 0,
 'has(z)': False}

Por fim, iremos criar uma lista contendo todos os outputs forneceidos pela função anterior para cada nome da lista Gender_names. Essa lista final, chamada de features_az, será utilizada para treinar e testar o nosso modelo, que irá utilizar o métoo Naive Bayes de classificação.

In [166]:
featuresets_az = [(gender_features_az(n), g) for (n,g) in Gender_names]
train_set, test_set = featuresets_az[500:], featuresets_az[:500]
classifier_az = nltk.NaiveBayesClassifier.train(train_set)
accuracy_az= nltk.classify.accuracy(classifier_az, test_set)
print("Acurácia do modelo: ", accuracy_az)

Acurácia do modelo:  0.77


A seguir, vamos testar o modelo para prever o gênero de cada membro do nosso grupo

In [167]:
membros = ['Henrique', 'Leticia', 'Lucas', 'Matheus', 'Milena']

[[i, classifier_az.classify(gender_features_az(i))] for i in membros]

[['Henrique', 'male'],
 ['Leticia', 'female'],
 ['Lucas', 'male'],
 ['Matheus', 'male'],
 ['Milena', 'female']]

## 3.1.3) Preenchimento do dataset e contagem da distribuição

De posse do modelo e da lista de colaboradores da organização, podemos descobrir como está a distribuição destes em gênero. Importante destacar que, quando mais balanceado for (cada gênero mais próximo de 50%), melhor avaliada será a organização

In [168]:
nomes_generos = [[i, classifier_az.classify(gender_features_az(i))] for i in nomes_ml['firstName'].to_list()]

In [169]:
generos= [g for (n,g) in nomes_generos]
males = generos.count('male')/len(generos)
females = generos.count('female')/len(generos)
print("Total employees: {}".format(len(generos)))
print("Total male: {}".format(males))
print("Total female: {}".format(females))

Total employees: 990
Total male: 0.37373737373737376
Total female: 0.6262626262626263


O nosso score para este quesito será dado pelo produto entre o percentual de mulheres e o percentual de homens da organização, dado que este valor é maximizado justamente quando ambos os valores são 50%, resultado, portanto, em 0.25 como valor máximo. 

In [170]:
score = (males*females)
score

0.2340577492092644

## 3.2) Americanas S.A.

### 3.2.1) Importação e limpeza dos dados de colaboradores 

De forma análoga, iremos novamente realizar o download da planilha contendo os 1.000 registros de funcionários da Americanas S.A., obtidos via web scrapping.

In [171]:
nomes_am = pd.read_csv("americanas.csv")
print("Shape: ", nomes_am.shape)
print("Colunas: ", nomes_am.columns)

Shape:  (968, 19)
Colunas:  Index(['profileUrl', 'fullName', 'firstName', 'lastName', 'profileImageUrl',
       'currentJob', 'pastJob', 'connectionDegree', 'job', 'location',
       'sharedConnections', 'commonConnection1', 'commonConnection2', 'url',
       'name', 'query', 'category', 'timestamp', 'error'],
      dtype='object')


In [172]:
nomes_am.head()

,profileUrl,fullName,firstName,lastName,profileImageUrl,currentJob,pastJob,connectionDegree,job,location,sharedConnections,commonConnection1,commonConnection2,url,name,query,category,timestamp,error
0,https://www.linkedin.com/in/andradetf,Tiago Andrade,Tiago,Andrade,https://media-exp1.licdn.com/dms/image/C4E03AQ...,NaN,NaN,2º,Data & Product,"Rio de Janeiro, Brasil","Thiago Marques (Pregador de Gauss, Fisher, Ber...",https://www.linkedin.com/in/ACoAABC2yqgBJrJ54O...,https://www.linkedin.com/in/ACoAABwNEZ4Bds3xI9...,https://www.linkedin.com/in/andradetf,Tiago Andrade,https://www.linkedin.com/search/results/people...,People,2021-09-26T19:05:59.856Z,NaN
1,https://www.linkedin.com/in/gabriel-marques-98...,Gabriel Marques,Gabriel,Marques,https://media-exp1.licdn.com/dms/image/C4E03AQ...,NaN,NaN,2º,Analista de operações na Americanas,"Nova Iguaçu, RJ",Rômulo Pinto Paiva Amaral é uma conexão em comum,https://www.linkedin.com/in/ACoAABwNEZ4Bds3xI9...,NaN,https://www.linkedin.com/in/gabriel-marques-98...,Gabriel Marques,https://www.linkedin.com/search/results/people...,People,2021-09-26T19:05:59.856Z,NaN
2,https://www.linkedin.com/in/lucas-gomes-44447a11a,Lucas Gomes,Lucas,Gomes,https://media-exp1.licdn.com/dms/image/C4E03AQ...,NaN,NaN,2º,Analista de inteligência de negócios | America...,"Rio de Janeiro, RJ","Thiago Marques (Pregador de Gauss, Fisher, Ber...",https://www.linkedin.com/in/ACoAABC2yqgBJrJ54O...,NaN,https://www.linkedin.com/in/lucas-gomes-44447a11a,Lucas Gomes,https://www.linkedin.com/search/results/people...,People,2021-09-26T19:05:59.856Z,NaN
3,https://www.linkedin.com/in/guilherme-moraes-1...,Guilherme Moraes,Guilherme,Moraes,https://media-exp1.licdn.com/dms/image/C4D03AQ...,NaN,NaN,2º,Desenvolvedor Backend,"São Paulo, SP","Thiago Marques (Pregador de Gauss, Fisher, Ber...",https://www.linkedin.com/in/ACoAABC2yqgBJrJ54O...,NaN,https://www.linkedin.com/in/guilherme-moraes-1...,Guilherme Moraes,https://www.linkedin.com/search/results/people...,People,2021-09-26T19:05:59.856Z,NaN
4,https://www.linkedin.com/in/yuri-lovalho,Yuri Lovalho,Yuri,Lovalho,https://media-exp1.licdn.com/dms/image/C4E03AQ...,NaN,NaN,2º,Product Owner | Americanas s.a.,"São Paulo, SP",NaN,NaN,NaN,https://www.linkedin.com/in/yuri-lovalho,Yuri Lovalho,https://www.linkedin.com/search/results/people...,People,2021-09-26T19:05:59.856Z,NaN


Também iremos separar apenas a coluna de interesse, contendo os nomes dos colaboradores e deletar os registros com valores nulos.

nomes_am = nomes_am[['firstName']].dropna()
print("Shape: ", nomes_am.shape)
nomes_am.head()

### 3.2.2) Preenchimento do dataset e contagem da distribuição

Dado que o modelo ja foi desenvolvido na etapa 3.1.2, basta apenas o aplicarmos para fazer o preenchimento da base de dados e realizarmos a análise da distribuição de gêneros.

In [175]:
nomes_generos = [[i, classifier_az.classify(gender_features_az(i))] for i in nomes_am['firstName'].to_list()]
generos= [g for (n,g) in nomes_generos]
males = generos.count('male')/len(generos)
females = generos.count('female')/len(generos)
print("Total employees: {}".format(len(generos)))
print("Total male: {}".format(males))
print("Total female: {}".format(females))

Total employees: 871
Total male: 0.34902411021814006
Total female: 0.6509758897818599


Assim, podemos calcular o score de diversidade para a Americanas S.A.

In [176]:
score = (males*females)
score

0.22720628070457569

# 4) Compilação das métricas e cálculo do score final

Na última etapa, iremos adicionar as métricas calculadas e as demais em um dataframe final e por fim calcular o score de cada organização. De forma compilada, temos que:

- **Magazine Luiza**:
    - Score de diversidade: 0.23
    - Nota do Glassdoor: 4.5 (fonte: https://www.glassdoor.com.br/Avalia%C3%A7%C3%B5es/Magazine-Luiza-Avalia%C3%A7%C3%B5es-E382606.htm)
    - Nota do ReclameAqui: 8.7 (fonte: https://www.reclameaqui.com.br/empresa/magazine-luiza-loja-online/)
    
- **Americanas**:
    - Score de diversidade: 0.22
    - Nota do Glassdoor: 3.8 (fonte: https://www.glassdoor.com.br/Vis%C3%A3o-geral/Trabalhar-na-americanas-s-a-EI_IE10939.13,27.htm)
    - Nota do ReclameAqui: 8.9 (fonte: https://www.reclameaqui.com.br/empresa/americanas-com-loja-online/)

In [191]:
df = pd.DataFrame({'Nome':['Magazine Luiza','Americana'],
                  'Diversidade':[0.23,0.22],
                  'Glassdoor':[4.5,3.8],
                  'ReclameAqui':[8.7,8.9]
                  })
df

,Nome,Diversidade,Glassdoor,ReclameAqui
0,Magazine Luiza,0.23,4.5,8.7
1,Americana,0.22,3.8,8.9


Vamos agora manipular as colunas para que ambas estejam sob uma mesma norma. Para a coluna 'Diversidade', o valor máximo a ser obtido é 0.25, logo iremos dividir todos os seus valores por este. O mesmo se aplica a coluna 'Glassdoor', cuja base é 5 e a coluna 'ReclameAqui', cuja base é 10.

In [192]:
df['Diversidade'] = (df['Diversidade']/0.25)*10
df['Glassdoor'] = (df['Glassdoor']/5)*10
df['ReclameAqui'] = (df['ReclameAqui']/10)*10

df

,Nome,Diversidade,Glassdoor,ReclameAqui
0,Magazine Luiza,9.2,9.0,8.7
1,Americana,8.8,7.6,8.9


Por fim, vamos criar uma nova coluna, chamada 'Score', que será a média aritmética das 3 anteriores. 

In [193]:
df['Score'] = ((df['Diversidade'] + df['Glassdoor'] +df['ReclameAqui'])/3)
df

,Nome,Diversidade,Glassdoor,ReclameAqui,Score
0,Magazine Luiza,9.2,9.0,8.7,8.966667
1,Americana,8.8,7.6,8.9,8.433333


In [196]:
df.to_csv('scores')

# 5) Considerações finais

Esse protótipo tem como objetivo trazer uma possível solução para tornarmos o ESG mais transparente e com menos greenwash. Hoje, os investidores estão mais criteriosos na hora de escolherem em quais organizações vão investir e, por isso, é importante que haja um acesso a informação que facilite o processo de tomada de decisão.

Como próximos passos, temos o objetivo de aumentar o número de métricas ESG, englobando fatores de Enviromental e Governance, além de tornar o processo mais automatizado. Tambem temos como objetivo desenvolver soluções que comprovem matematicamente como métricas ESG altas estão correlacionadas com a perspectiva de valuation futuro das organizações, tornando assim, mais fácil para a comunidade compr